In [0]:
import requests
import os
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime

In [0]:
log_messages = []
log_messages.append(f"{datetime.now()} - INFO - Iniciando ingestão")


In [0]:
load_dotenv()

API_KEY = os.getenv("API_KEY")
API_URL = os.getenv("API_URL")
DEFAULT_PATH = os.getenv("DEFAULT_PATH")
LOG_PATH = os.getenv("LOG_PATH")

In [0]:
_rate = dbutils.widgets.get('rate')
log_messages.append(f"{datetime.now()} - INFO - Parametro utilizado no Rate: {_rate}")

In [0]:
def get_rates():
  url = f'{API_URL}{API_KEY}/latest/{_rate}'
  response = requests.get(url)
  log_messages.append(f"{datetime.now()} - INFO - Request para {url}")
  if response.status_code != 200:
    log_messages.append(f"{datetime.now()} - ERROR - Erro ao realizar request para {url}")
  else:
    log_messages.append(f"{datetime.now()} - INFO - Sucesso ao realizar request para {url}")
  return response.json()

def genarate_df():
  data = get_rates()
  rates = data.get('conversion_rates', {})
  log_messages.append(f"{datetime.now()} - INFO - DataFrame gerado")
  df = (
        pd.DataFrame(rates, index=[0])
        .melt(var_name='currency', value_name='value')
        )
  log_messages.append(f"{datetime.now()} - INFO - Dados do DataFrame carregados")
  return df

def save_df_to_json(df, path=DEFAULT_PATH):
    current_date = datetime.today().strftime('%Y-%m-%d')
    file_path = f"{path}/Raw/{current_date}.json"
    df.to_json(file_path, orient='records', lines=True)
    


In [0]:

df = genarate_df()
save_df_to_json(df)
log_messages.append(f"{datetime.now()} - INFO - Daframe salvo no S3 e log salvo")
log_text = "\n".join(log_messages)
dbutils.fs.put(f"{LOG_PATH}/ingestao.log", log_text, overwrite=True)

